In [1]:
import os
os.chdir('/Users/layne/Desktop/aiqc')
os.getcwd()

'/Users/layne/Desktop/aiqc'

In [2]:
import aiqc
from aiqc import examples

/Users/layne/.pyenv/versions/3.7.6/envs/jupyterlab/lib/python3.7/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import *

In [4]:
from importlib import reload
aiqc.delete_db(True)
reload(aiqc)
aiqc.create_db()


=> Success - deleted database file at path:
/Users/layne/Library/Application Support/aiqc/aiqc.sqlite3


=> Success - created database file for machine learning metrics at path:
/Users/layne/Library/Application Support/aiqc/aiqc.sqlite3


=> Success - created the following tables within database:
['algorithm', 'batch', 'dataset', 'encoderset', 'featureset', 'file', 'fold', 'foldset', 'hyperparamcombo', 'hyperparamset', 'image', 'job', 'jobset', 'label', 'labelcoder', 'result', 'splitset', 'tabular']



---

In [5]:
dataset = aiqc.Dataset.Tabular.from_path(
    examples.get_demo_file_path('dtype_testing.parquet')
    , source_file_format = 'parquet'
)

In [6]:
dataset.Tabular.get_main_tabular(dataset.id).columns

['doom_float64',
 'radiance_float32',
 'temperature_float',
 'weekday_object',
 'hour_object',
 'moonday_int64',
 'month_int32',
 'color_uint8',
 'shape_int',
 'kingdom_category']

In [7]:
label = dataset.make_label(columns=['kingdom_category'])

In [8]:
featureset = dataset.make_featureset(exclude_columns=['kingdom_category'])

In [9]:
splitset = featureset.make_splitset(
    label_id=label.id
    , size_test = 0.22
    , size_validation = 0.12 
)

---

In [10]:
encoderset = splitset.make_encoderset()

In [11]:
labelcoder = encoderset.make_labelcoder(
    sklearn_preprocess = StandardScaler()
    , only_fit_train = False
)

ValueError: 
Yikes - Encoder failed to fit the columns you filtered.

Either the data is dirty (e.g. contains NaNs),
or you used one of the incompatible combinations of data type and encoder seen below :

{'string': ['KBinsDiscretizer', 'KernelCenterer', 'MaxAbsScaler', 'MinMaxScaler', 'PowerTransformer', 'QuantileTransformer', 'RobustScaler', 'StandardScaler'], 'float': ['LabelBinarizer'], 'numeric array without dimensions both odd and square. E.g. 3x3, 5x5': ['KernelCenterer']}


^ this should warn